<a href="https://colab.research.google.com/github/cbikash/MREs-workshops/blob/main/Workshop%20%233.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tasks

## P 3.1: Download the CarSharing dataset from Canvas and apply the normal linear regression and the Ridge regression methods to predict the 'demand' column. Use 5 different values for the regularisation parameter in the Ridge regression and repeat the prediction. Calculate all evaluation metrics discussed in the lecture session for both models and all repetitions and print them. Interpret the results and report your interpretation in the report cell (completing the report cell is required) (15%)

In [23]:
############# WRITE YOUR CODE IN THIS CELL (IF APPLICABLE)  ####################
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
data = pd.read_csv("https://raw.githubusercontent.com/cbikash/MREs-workshops/main/CarSharing.csv")
data.drop(columns=["id", "timestamp"], inplace=True)
data


,season,holiday,workingday,weather,temp,temp_feel,humidity,windspeed,demand
0,spring,No,No,Clear or partly cloudy,9.84,14.395,81.0,0.0000,2.772589
1,spring,No,No,Clear or partly cloudy,9.02,13.635,80.0,0.0000,3.688879
2,spring,No,No,Clear or partly cloudy,9.02,13.635,80.0,0.0000,3.465736
3,spring,No,No,Clear or partly cloudy,9.84,14.395,75.0,0.0000,2.564949
4,spring,No,No,Clear or partly cloudy,9.84,14.395,75.0,0.0000,0.000000
...,...,...,...,...,...,...,...,...,...
8703,fall,No,No,Clear or partly cloudy,30.34,34.850,70.0,19.0012,5.030438
8704,fall,No,No,Clear or partly cloudy,30.34,34.850,70.0,16.9979,4.465908
8705,fall,No,No,Clear or partly cloudy,30.34,34.850,70.0,19.9995,4.290459
8706,fall,No,No,Clear or partly cloudy,29.52,34.850,74.0,16.9979,3.713572


In [24]:
#data prepration
data.info()
data.describe()
data.head()
weather = data['weather'].value_counts()
print(weather)
print("\n")
print(data.isnull().sum())

# filling null with mean value for number and mode for string type of data
data.fillna({
    "temp" : data["temp"].mean(),
    "temp_feel": data["temp_feel"].mean(),
    "windspeed" : data["windspeed"].mean(),
    "humidity" : data['humidity'].mean()
}, inplace=True)
print(data.isnull().sum())
print(data.isnull())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8708 entries, 0 to 8707
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      8708 non-null   object 
 1   holiday     8708 non-null   object 
 2   workingday  8708 non-null   object 
 3   weather     8708 non-null   object 
 4   temp        7506 non-null   float64
 5   temp_feel   8606 non-null   float64
 6   humidity    8669 non-null   float64
 7   windspeed   8508 non-null   float64
 8   demand      8708 non-null   float64
dtypes: float64(5), object(4)
memory usage: 612.4+ KB
weather
Clear or partly cloudy               5858
Mist                                 2142
Light snow or rain                    707
heavy rain/ice pellets/snow + fog       1
Name: count, dtype: int64


season           0
holiday          0
workingday       0
weather          0
temp          1202
temp_feel      102
humidity        39
windspeed      200
demand           0
dtype: int64
s

,season,holiday,workingday,weather,temp,temp_feel,humidity,windspeed,demand
0,spring,No,No,Clear or partly cloudy,9.84,14.395,81.0,0.0,2.772589
1,spring,No,No,Clear or partly cloudy,9.02,13.635,80.0,0.0,3.688879
2,spring,No,No,Clear or partly cloudy,9.02,13.635,80.0,0.0,3.465736
3,spring,No,No,Clear or partly cloudy,9.84,14.395,75.0,0.0,2.564949
4,spring,No,No,Clear or partly cloudy,9.84,14.395,75.0,0.0,0.000000


In [25]:
# converting categorical values into numberical value using pandas get_dummies()
df_encoded = pd.get_dummies(data, columns=['season', 'weather'], drop_first=True)
df_encoded.head()

#label encoder is better for bolean value
le = LabelEncoder()
df_encoded[['holiday', 'workingday']] = df_encoded[['holiday', 'workingday']].apply(lambda col: le.fit_transform(col))



In [26]:
df_encoded

,holiday,workingday,temp,temp_feel,humidity,windspeed,demand,season_spring,season_summer,season_winter,weather_Light snow or rain,weather_Mist,weather_heavy rain/ice pellets/snow + fog
0,0,0,9.84,14.395,81.0,0.0000,2.772589,True,False,False,False,False,False
1,0,0,9.02,13.635,80.0,0.0000,3.688879,True,False,False,False,False,False
2,0,0,9.02,13.635,80.0,0.0000,3.465736,True,False,False,False,False,False
3,0,0,9.84,14.395,75.0,0.0000,2.564949,True,False,False,False,False,False
4,0,0,9.84,14.395,75.0,0.0000,0.000000,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8703,0,0,30.34,34.850,70.0,19.0012,5.030438,False,False,False,False,False,False
8704,0,0,30.34,34.850,70.0,16.9979,4.465908,False,False,False,False,False,False
8705,0,0,30.34,34.850,70.0,19.9995,4.290459,False,False,False,False,False,False
8706,0,0,29.52,34.850,74.0,16.9979,3.713572,False,False,False,False,False,False


In [27]:

scaler = StandardScaler()
# numerical_cols = ['temp', 'temp_feel', 'humidity', 'windspeed']
# df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

X = df_encoded.drop('demand', axis=1)
y = df_encoded['demand']
#converting data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [28]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [29]:
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(X_train,y_train)

Ridge()

In [30]:
y_pred_ridge1= ridge.predict(X_test)

In [31]:
y_pred = model.predict(X_test)
print(y_pred)

[3.35295606 5.45392565 4.1826903  ... 5.28886285 5.18842949 4.96104378]


In [32]:

from sklearn.metrics import mean_squared_error, r2_score

# Calculate and print R^2 score.
r2 = r2_score(y_test, y_pred)
print(f"R-squared normal regression: {r2:.4f}")

R-squared normal regression: 0.3107


In [33]:
#Evaluating ridge model performance using the coefficient of determination is easy with scikit-learn.

from sklearn.metrics import mean_squared_error, r2_score

# Calculate and print R^2 score.
r2 = r2_score(y_test, y_pred_ridge1)
print(f"R-squared ridge: {r2:.4f}")

R-squared ridge: 0.3107
